In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of the website
URL = "https://www.redbus.in/online-booking/chandigarh-transport-undertaking-ctu"

# Initialize WebDriver globally
driver = None

def initialize_driver():
    global driver
    driver = webdriver.Chrome()
    driver.maximize_window()

def load_page(url):
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

# Function to scrape bus routes
def scrape_bus_routes():
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details
def scrape_bus_details(url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  # Allow the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages():
    for page in range(1, 6):  # There are 5 pages
        try:
            load_page(URL)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes()
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)

        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

# Main execution
initialize_driver()
try:
    scrape_all_pages()

    # Convert the list of dictionaries to a DataFrame
    df = pd.DataFrame(all_bus_details)

    # Save the DataFrame to a CSV file
    df.to_csv('chandigarh_bus_details.csv', index=False)

finally:
    if driver:
        driver.quit()


Error occurred while scraping bus details for https://www.redbus.in/bus-tickets/chandigarh-to-hamirpur-himachal-pradesh: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7A8FE6CB5+28821]
	(No symbol) [0x00007FF7A8F53840]
	(No symbol) [0x00007FF7A8DF578A]
	(No symbol) [0x00007FF7A8E491BE]
	(No symbol) [0x00007FF7A8E494AC]
	(No symbol) [0x00007FF7A8E92647]
	(No symbol) [0x00007FF7A8E6F33F]
	(No symbol) [0x00007FF7A8E8F412]
	(No symbol) [0x00007FF7A8E6F0A3]
	(No symbol) [0x00007FF7A8E3A778]
	(No symbol) [0x00007FF7A8E3B8E1]
	GetHandleVerifier [0x00007FF7A931FCAD+3408013]
	GetHandleVerifier [0x00007FF7A933741F+3504127]
	GetHandleVerifier [0x00007FF7A932B5FD+3455453]
	GetHandleVerifier [0x00007FF7A90ABDBB+835995]
	(No symbol) [0x00007FF7A8F5EB5F]
	(No symbol) [0x00007FF7A8F5A814]
	(No symbol) [0x00007FF7A8F5A9AD]
	(No symbol) [0x00007FF7A8F4A199]
	BaseThreadInitThunk [0x00007FFCF153259D+29]
	RtlUserThreadStart [0x00007FFCF27AAF38+40]

Error occurred while scraping bus details for https://

In [2]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Delhi to Chandigarh,https://www.redbus.in/bus-tickets/delhi-to-cha...,HRTC - 513,Ordinary 3+2 Non AC Seater,10:30,04h 30m,15:00,1.7,INR 290,27 Seats available
1,Delhi to Chandigarh,https://www.redbus.in/bus-tickets/delhi-to-cha...,HRTC - 514,Ordinary 3+2 Non AC Seater,11:30,06h 10m,17:40,3.9,INR 290,37 Seats available
2,Delhi to Chandigarh,https://www.redbus.in/bus-tickets/delhi-to-cha...,HRTC - 6754,Ordinary 3+2 Non AC Seater,12:00,05h 00m,17:00,2.9,INR 300,41 Seats available
3,Delhi to Chandigarh,https://www.redbus.in/bus-tickets/delhi-to-cha...,HRTC - 551,A/C Executive (2+3),15:30,05h 55m,21:25,3.9,INR 400,42 Seats available
4,Delhi to Chandigarh,https://www.redbus.in/bus-tickets/delhi-to-cha...,HRTC - 525,Ordinary 3+2 Non AC Seater,16:50,05h 50m,22:40,3.9,INR 290,27 Seats available
...,...,...,...,...,...,...,...,...,...,...
192,Chandigarh to Haridwar,https://www.redbus.in/bus-tickets/chandigarh-t...,Chandigarh Transport Undertaking (CTU) - 165962,HVAC Seater (2+3),13:40,05h 50m,19:30,3.9,INR 447.6,43 Seats available
193,Chandigarh to Haridwar,https://www.redbus.in/bus-tickets/chandigarh-t...,HRTC - 6704,Ordinary 3+2 Non AC Seater,17:30,06h 45m,00:15,3.9,INR 314,37 Seats available
194,Chandigarh to Haridwar,https://www.redbus.in/bus-tickets/chandigarh-t...,Vijay Tour and Travels,Bharat Benz A/C Seater /Sleeper (2+1),21:30,05h 10m,02:40,3.5,INR 1249,9 Seats available
195,Chandigarh to Haridwar,https://www.redbus.in/bus-tickets/chandigarh-t...,,,,,,,,
